In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder

plt.style.use('ggplot')
warnings.filterwarnings(action='ignore')
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import norm
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.ar_model import AR
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import numpy as np
import matplotlib.pyplot as plt
import statsmodels

In [2]:
def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    '''값이 1개 존재하면 제거'''
    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   

def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    # qualitative to quantitative
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    print('Done.')
    return train,test

In [43]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [44]:
train, test = del_columns(train,test)

In [45]:
trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

In [46]:
trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

In [7]:
# trainA_31, testA_31 = del_columns_2(trainA_31,testA_31)
# train_T_31, test_T_31 = del_columns_2(train_T_31,test_T_31)
# train_O_31, test_O_31 = del_columns_2(train_O_31,test_O_31)

In [47]:
trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

In [48]:
trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

Done.
Done.
Done.


In [49]:
trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

In [50]:
def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

In [51]:
import math
def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

In [52]:
drop_a = ['X_1167','X_1168','X_1169','X_1170',
          'X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']

In [53]:
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

In [54]:
trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)

In [55]:
trainA_31_x,testA_31_x = multi_col(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = multi_col(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = multi_col(train_O_31_x,test_O_31_x)

In [56]:
trainA_31_x,testA_31_x = log(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = log(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = log(train_O_31_x,test_O_31_x)

In [57]:
print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 5566) (67, 5566) (343, 1618) (239, 1618)


In [58]:
from catboost import *

In [59]:
model = CatBoostRegressor(random_state=4321,verbose=500,iterations=3500,learning_rate=0.05)

In [60]:
model.fit(trainA_31_x, trainA_31_y)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0098366	total: 174ms	remaining: 10m 9s
500:	learn: 0.0001915	total: 1m 29s	remaining: 8m 57s
1000:	learn: 0.0000060	total: 2m 52s	remaining: 7m 9s
1500:	learn: 0.0000002	total: 4m 40s	remaining: 6m 14s
2000:	learn: 0.0000000	total: 6m	remaining: 4m 30s
2500:	learn: 0.0000000	total: 7m 14s	remaining: 2m 53s
3000:	learn: 0.0000000	total: 8m 24s	remaining: 1m 23s
3499:	learn: 0.0000000	total: 9m 28s	remaining: 0us


In [61]:
model.fit(train_T_31_x, train_T_31_y)
pred_t = model.predict(test_T_31_x)

0:	learn: 0.0047113	total: 48.5ms	remaining: 2m 49s
500:	learn: 0.0001497	total: 21.2s	remaining: 2m 6s
1000:	learn: 0.0000059	total: 44.3s	remaining: 1m 50s
1500:	learn: 0.0000003	total: 1m 11s	remaining: 1m 35s
2000:	learn: 0.0000000	total: 1m 41s	remaining: 1m 15s
2500:	learn: 0.0000000	total: 2m 2s	remaining: 49s
3000:	learn: 0.0000000	total: 2m 24s	remaining: 24s
3499:	learn: 0.0000000	total: 2m 45s	remaining: 0us


In [62]:
model.fit(train_O_31_x, train_O_31_y)#, sample_weight = train_O_31_w['Y_Class'].values)
pred_o = model.predict(test_O_31_x)

0:	learn: 0.0031555	total: 16.8ms	remaining: 58.7s
500:	learn: 0.0000055	total: 9.38s	remaining: 56.2s
1000:	learn: 0.0000000	total: 17s	remaining: 42.5s
1500:	learn: 0.0000000	total: 22.3s	remaining: 29.7s
2000:	learn: 0.0000000	total: 27.6s	remaining: 20.7s
2500:	learn: 0.0000000	total: 32.7s	remaining: 13.1s
3000:	learn: 0.0000000	total: 37.4s	remaining: 6.22s
3499:	learn: 0.0000000	total: 42.4s	remaining: 0us


In [67]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('0214.csv',index=False)

In [68]:
df = pd.read_csv('0214.csv') # 

In [148]:
df = pd.read_csv('multiLOG.csv')

In [426]:
df = pd.read_csv('adf9993_it2000_s2784.csv')

In [35]:
df = pd.read_csv('adf9993.csv')

In [71]:
st = pd.read_csv('SotaToCha_0.csv')

In [328]:
ct = pd.read_csv('캣2_5.csv')

In [146]:
li = pd.read_csv('sub0205_cat_reg_split_line.csv')

In [147]:
li['Y_Class'].value_counts()

1    269
0     32
2      9
Name: Y_Class, dtype: int64

In [145]:
st['Y_Class'].value_counts()

1    262
0     43
2      5
Name: Y_Class, dtype: int64

In [329]:
ct['Y_Class'].value_counts()

1    275
0     30
2      5
Name: Y_Class, dtype: int64

In [128]:
df['Y_Class'].value_counts()

1    260
0     37
2     13
Name: Y_Class, dtype: int64

In [38]:
df['Y_Class'].value_counts()/len(df)

1    0.864516
0    0.106452
2    0.029032
Name: Y_Class, dtype: float64

In [41]:
df['Y_Class'].value_counts()

1    271
0     31
2      8
Name: Y_Class, dtype: int64

In [77]:
df['Y_Class'].value_counts()

1    270
0     34
2      6
Name: Y_Class, dtype: int64

In [79]:
df[df['PRODUCT_ID']=='TEST_011'].index

Int64Index([11], dtype='int64')

In [81]:
df.iloc[11]['Y_Class']=1

In [82]:
df.iloc[11]

PRODUCT_ID    TEST_011
Y_Class              2
Name: 11, dtype: object

In [76]:
df[df['PRODUCT_ID']=='TEST_011']['Y_Class']=1

In [66]:
df[df['Y_Class']==2]['PRODUCT_ID'].unique()

array(['TEST_011', 'TEST_038', 'TEST_078', 'TEST_160', 'TEST_161',
       'TEST_183'], dtype=object)

In [72]:
st[st['Y_Class']==2]['PRODUCT_ID'].unique()

array(['TEST_038', 'TEST_078', 'TEST_160', 'TEST_161', 'TEST_183'],
      dtype=object)

In [37]:
df[df['Y_Class']==2]['PRODUCT_ID'].unique()

array(['TEST_038', 'TEST_078', 'TEST_095', 'TEST_160', 'TEST_161',
       'TEST_172', 'TEST_176', 'TEST_183', 'TEST_204'], dtype=object)

In [352]:
df[df['Y_Class']==0]['PRODUCT_ID'].unique()

array(['TEST_003', 'TEST_007', 'TEST_008', 'TEST_013', 'TEST_014',
       'TEST_035', 'TEST_037', 'TEST_039', 'TEST_041', 'TEST_042',
       'TEST_047', 'TEST_049', 'TEST_051', 'TEST_052', 'TEST_053',
       'TEST_054', 'TEST_058', 'TEST_060', 'TEST_061', 'TEST_064',
       'TEST_065', 'TEST_066', 'TEST_071', 'TEST_086', 'TEST_088',
       'TEST_250', 'TEST_251', 'TEST_252', 'TEST_254', 'TEST_284',
       'TEST_285', 'TEST_292', 'TEST_293'], dtype=object)

In [33]:
st[st['Y_Class']==2]['PRODUCT_ID'].unique()

array(['TEST_038', 'TEST_078', 'TEST_160', 'TEST_161', 'TEST_183'],
      dtype=object)

In [331]:
ct[ct['Y_Class']==0]['PRODUCT_ID'].unique()

array(['TEST_003', 'TEST_007', 'TEST_008', 'TEST_013', 'TEST_014',
       'TEST_035', 'TEST_037', 'TEST_039', 'TEST_041', 'TEST_042',
       'TEST_047', 'TEST_049', 'TEST_051', 'TEST_052', 'TEST_053',
       'TEST_054', 'TEST_055', 'TEST_058', 'TEST_060', 'TEST_061',
       'TEST_062', 'TEST_063', 'TEST_064', 'TEST_065', 'TEST_066',
       'TEST_071', 'TEST_086', 'TEST_088', 'TEST_285', 'TEST_292'],
      dtype=object)

In [44]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('weight0_5.csv',index=False)

# scroe -> 0.65

In [45]:
df=pd.read_csv('weight0_5.csv')

In [94]:
df = pd.read_csv('fi.csv')

In [46]:
len(df[df['Y_Class']==0])

40

In [47]:
len(df[df['Y_Class']==1])

267

In [48]:
len(df[df['Y_Class']==2])

3

In [25]:
testA_31['Y_Class'].values

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1], dtype=int64)

In [49]:
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0], dtype=int64)

In [51]:
test_T_31['Y_Class'].values

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [30]:
model.get_feature_importance()

array([0.00000000e+00, 0.00000000e+00, 8.86700035e-03, ...,
       9.26632888e-04, 5.97557376e-05, 2.07225631e-02])

In [144]:
high_a = A_importance[A_importance['feature_importance']>1]['feature_names'].unique()

In [145]:
high_t = T_importance[T_importance['feature_importance']>1]['feature_names'].unique()

In [146]:
high_o = O_importance[O_importance['feature_importance']>1]['feature_names'].unique()

# ver 2 - score :

#### make feature

In [125]:
y1=train[train['Y_Class']==1]

In [230]:
for k in high_a :
    trainA_31_x[k+'_box'] = 0
    testA_31_x[k+'_box'] = 0
    p25 = y1[k].quantile(q=0.25, interpolation='nearest')
    p75 = y1[k].quantile(q=0.75, interpolation='nearest')
    trainA_31_x.loc[(trainA_31_x[k] >=p25) & (trainA_31_x[k] < p75), k+'_box'] = 1
    testA_31_x.loc[(testA_31_x[k] >=p25) & (testA_31_x[k] < p75), k+'_box'] = 1

In [231]:
for k in high_t :
    train_T_31_x[k+'_box'] = 0
    test_T_31_x[k+'_box'] = 0
    p25 = y1[k].quantile(q=0.25, interpolation='nearest')
    p75 = y1[k].quantile(q=0.75, interpolation='nearest')
    train_T_31_x.loc[(train_T_31_x[k] >=p25) & (train_T_31_x[k] < p75), k+'_box'] = 1
    test_T_31_x.loc[(test_T_31_x[k] >=p25) & (test_T_31_x[k] < p75), k+'_box'] = 1

In [232]:
for k in high_o :
    train_O_31_x[k+'_box'] = 0
    test_O_31_x[k+'_box'] = 0
    p25 = y1[k].quantile(q=0.25, interpolation='nearest')
    p75 = y1[k].quantile(q=0.75, interpolation='nearest')
    train_O_31_x.loc[(train_O_31_x[k] >=p25) & (train_O_31_x[k] < p75), k+'_box'] = 1
    test_O_31_x.loc[(test_O_31_x[k] >=p25) & (test_O_31_x[k] < p75), k+'_box'] = 1

In [233]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=2000,learning_rate=0.033)
model.fit(trainA_31_x, trainA_31_y)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0098580	total: 128ms	remaining: 4m 15s
500:	learn: 0.0006622	total: 27.9s	remaining: 1m 23s
1000:	learn: 0.0000697	total: 52.3s	remaining: 52.2s
1500:	learn: 0.0000068	total: 1m 17s	remaining: 25.9s
1999:	learn: 0.0000006	total: 1m 44s	remaining: 0us


In [234]:
model.fit(train_T_31_x, train_T_31_y)
pred_t = model.predict(test_T_31_x)

0:	learn: 0.0047185	total: 58ms	remaining: 1m 56s
500:	learn: 0.0005000	total: 11.8s	remaining: 35.2s
1000:	learn: 0.0000569	total: 23.7s	remaining: 23.7s
1500:	learn: 0.0000062	total: 35.5s	remaining: 11.8s
1999:	learn: 0.0000008	total: 47.3s	remaining: 0us


In [235]:
model.fit(train_O_31_x, train_O_31_y)
pred_o = model.predict(test_O_31_x)

0:	learn: 0.0031689	total: 4.59ms	remaining: 9.17s
500:	learn: 0.0000475	total: 3.33s	remaining: 9.97s
1000:	learn: 0.0000008	total: 6.34s	remaining: 6.33s
1500:	learn: 0.0000000	total: 8.87s	remaining: 2.95s
1999:	learn: 0.0000000	total: 12.2s	remaining: 0us


In [236]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('fe_ver2.csv',index=False)

# scroe -> 0.65

In [237]:
df = pd.read_csv('fe_ver2.csv')

In [238]:
len(df[df['Y_Class']==0])

27

In [239]:
len(df[df['Y_Class']==2])

6

In [184]:
df[df['Y_Class']==2]

,PRODUCT_ID,Y_Class
12,TEST_012,2
38,TEST_038,2
78,TEST_078,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
183,TEST_183,2


In [189]:
df[df['Y_Class']==0]

,PRODUCT_ID,Y_Class
3,TEST_003,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
35,TEST_035,0
37,TEST_037,0
39,TEST_039,0
47,TEST_047,0
49,TEST_049,0
51,TEST_051,0


In [187]:
df[df['Y_Class']==0]

,PRODUCT_ID,Y_Class
3,TEST_003,0
7,TEST_007,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
35,TEST_035,0
37,TEST_037,0
39,TEST_039,0
41,TEST_041,0
42,TEST_042,0


In [134]:
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1], dtype=int64)

In [185]:
df = pd.read_csv('캣2_5.csv')

# ver3

In [209]:
y1=train[train['Y_Class']==1]
y0=train[train['Y_Class']==0]
y2=train[train['Y_Class']==2]

In [205]:
y0['Y_Quality'].max()

0.525066667

In [206]:
y1['Y_Quality'].min()

0.525085714

In [207]:
y1['Y_Quality'].max()

0.534842857

In [208]:
y2['Y_Quality'].min()

0.534950794

In [213]:
y0['Y_Quality'].quantile(q=0.8, interpolation='nearest')

0.5244190479999999

In [215]:
y1['Y_Quality'].quantile(q=0.2, interpolation='nearest')

0.52797619

In [216]:
y1['Y_Quality'].quantile(q=0.8, interpolation='nearest')

0.5325063489999999

In [214]:
y2['Y_Quality'].quantile(q=0.2, interpolation='nearest')

0.535965079

In [218]:
train[((train['Y_Quality']<0.535965079)&(train['Y_Quality']>0.5325063489999999)) | ((train['Y_Quality']>0.5244190479999999)&(train['Y_Quality']<0.52797619))].to_csv('사이값.csv',index=False)

In [220]:
ndf = train[((train['Y_Quality']<0.535965079)&(train['Y_Quality']>0.5325063489999999)) | ((train['Y_Quality']>0.5244190479999999)&(train['Y_Quality']<0.52797619))]

In [224]:
len(ndf[ndf['PRODUCT_CODE']=='A_31'])

72

In [225]:
len(ndf[ndf['PRODUCT_CODE']=='T_31'])

122

In [226]:
len(ndf[ndf['PRODUCT_CODE']=='O_31'])

4

In [228]:
trainA_31.columns

Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
       'PRODUCT_CODE', 'X_128', 'X_129', 'X_130', 'X_131',
       ...
       'X_2861', 'X_2862', 'X_2863', 'X_2864', 'X_2865', 'X_2866', 'X_2867',
       'X_2868', 'X_2869', 'X_2870'],
      dtype='object', length=1871)

In [229]:
ndf[ndf['PRODUCT_CODE']=='A_31'][trainA_31.columns]

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_128,X_129,X_130,X_131,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,7813.0,7813.0,NaN,NaN,...,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77
6,TRAIN_006,1,0.533665,2022-06-13 6:03,T050304,A_31,7820.0,7820.0,NaN,NaN,...,188.180000,176.9,384.0,368.425926,353.0,32.50,41.42,38.36,30.83,76.93
12,TRAIN_012,1,0.533632,2022-06-13 6:52,T050304,A_31,7827.0,7827.0,NaN,NaN,...,175.940000,162.4,392.0,374.148148,352.0,36.55,32.32,54.25,35.15,78.86
16,TRAIN_016,1,0.534054,2022-06-13 7:25,T050304,A_31,7832.0,7832.0,NaN,NaN,...,176.465517,165.4,384.0,368.962264,353.0,40.53,31.70,49.49,35.73,79.75
33,TRAIN_033,1,0.526656,2022-06-21 7:14,T010305,A_31,NaN,NaN,NaN,NaN,...,168.258621,158.3,446.0,429.301887,413.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,TRAIN_550,1,0.533695,2022-08-26 8:26,T050307,A_31,NaN,NaN,2278.0,2278.0,...,177.033333,161.4,383.0,363.037037,342.0,52.61,53.10,57.30,56.59,66.83
551,TRAIN_551,1,0.525271,2022-08-26 9:38,T050307,A_31,NaN,NaN,2407.0,2407.0,...,187.772414,168.7,384.0,369.433962,352.0,52.28,56.24,54.47,50.56,62.53
553,TRAIN_553,0,0.524686,2022-08-26 11:23,T010305,A_31,NaN,NaN,NaN,NaN,...,191.286207,181.7,466.0,444.924528,424.0,NaN,NaN,NaN,NaN,NaN
573,TRAIN_573,1,0.526948,2022-09-03 20:03,T050304,A_31,9401.0,9401.0,NaN,NaN,...,189.210345,167.5,383.0,370.528302,352.0,52.65,58.95,41.01,58.43,64.46


In [ ]:
trainA_31.columns

In [19]:
93*2 - 93*0.26*2*3

40.920000000000016

In [20]:
41/(2-0.26*3)

33.60655737704918

In [18]:
93-79

14

In [14]:
2-0.26

1.74